<a href="https://colab.research.google.com/github/Kostrykina18/NLP_HSE_4year/blob/main/NNm_hw4_kostrykina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ДЗ4
## Екатерина Кострыкина БКЛ181

In [1]:
!pip install -q -U watermark

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 2.1.3 which is incompatible.


In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.4 MB/s 
     |████████████████████████████████| 895 kB 41.8 MB/s 
     |████████████████████████████████| 6.6 MB 35.9 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 5.5.0

numpy       : 1.21.5
pandas      : 1.3.5
torch       : 1.10.0+cu111
transformers: 4.18.0



In [5]:
! pip install datasets

     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 46.3 MB/s 
     |████████████████████████████████| 136 kB 33.6 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |████████████████████████████████| 127 kB 49.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 49.9 MB/s 
     |████████████████████████████████| 271 kB 49.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [1]:
from datasets import load_dataset
data = load_dataset('imdb', split={'train': 'train[:1000]+train[-1000:]',
                                   'test': 'test[:500]+test[-500:]',
                                   'validation': 'test[500:1000]+test[-1000:-500]'}
                    )

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [3]:
import transformers
from transformers import BertModel, AutoTokenizer, BertTokenizer, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F

import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device = torch.device('cuda:0')

In [5]:
MODEL_NAME = 'prajjwal1/bert-small'

In [6]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [7]:
train_texts = data['train']['text']
val_texts = data['validation']['text']
test_texts = data['test']['text']

train_labels = data['train']['label']
val_labels = data['validation']['label']
test_labels = data['test']['label']

In [8]:
train_tokenized = tokenizer(train_texts, 
                          truncation=True,
                          padding=True,
                          return_token_type_ids=False,
                          return_attention_mask=True,
                          max_length=512
                          )

val_tokenized = tokenizer(val_texts,
                        truncation=True,
                        padding=True,
                        return_token_type_ids=False,
                        return_attention_mask=True,
                        max_length=512
                        )

test_tokenized = tokenizer(test_texts,
                         truncation=True,
                         padding=True, 
                         return_token_type_ids=False,
                         return_attention_mask=True, 
                         max_length=512
                         )

In [9]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_tokenized, train_labels)
val_dataset = IMDbDataset(val_tokenized, val_labels)
test_dataset = IMDbDataset(test_tokenized, test_labels)

Функция для подсчета метрик

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Зададим параметры обучения с помощью TrainingArguments (одинаковые для всех моделей)


In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=5,
    evaluation_strategy="epoch"
    )

##обучите ту модель, которую мы разбирали на занятии (класс SentimentClassifier из тетрадки) -- 2 балла

Чтобы обучать модель с помощью Trainer, класс должен возвращать лосс и предсказания. Добавим подсчет лосса к классу с семинара

In [12]:
loss_func = nn.CrossEntropyLoss().to(device)

In [13]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    self.bert = BertModel.from_pretrained(MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask, labels):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    output = self.drop(pooled_output)
    return (loss_func(self.out(output), labels), self.out(output))

In [14]:
set(train_labels)

{0, 1}

In [15]:
model = SentimentClassifier(2)
model = model.to(device)

Downloading:   0%|          | 0.00/111M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
data_collator = DataCollatorWithPadding(tokenizer)

Начнем обучение

In [17]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics,    # metrics to evaluate
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.269900,0.483926,0.774000,0.800353,0.716772,0.906000
2,0.287900,0.391909,0.839000,0.837209,0.846626,0.828000
3,0.091800,0.467238,0.843000,0.845015,0.834308,0.856000
4,0.161800,0.547010,0.851000,0.846233,0.874200,0.820000
5,0.111900,0.571475,0.846000,0.845691,0.847390,0.844000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1250, training_loss=0.2800739691585302, metrics={'train_runtime': 436.5301, 'train_samples_per_second': 22.908, 'train_steps_per_second': 2.863, 'total_flos': 0.0, 'train_loss': 0.2800739691585302, 'epoch': 5.0})

Результаты на валидационной выборке

In [18]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.846,
 'eval_f1': 0.845691382765531,
 'eval_loss': 0.571475088596344,
 'eval_precision': 0.8473895582329317,
 'eval_recall': 0.844,
 'eval_runtime': 13.0897,
 'eval_samples_per_second': 76.396,
 'eval_steps_per_second': 9.55}

Результаты на тестовой выборке

In [19]:
trainer.evaluate(eval_dataset=test_dataset, 
                 metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 5.0,
 'test_accuracy': 0.862,
 'test_f1': 0.8608870967741936,
 'test_loss': 0.5069182515144348,
 'test_precision': 0.8678861788617886,
 'test_recall': 0.854,
 'test_runtime': 13.0566,
 'test_samples_per_second': 76.59,
 'test_steps_per_second': 9.574}

##измените модель, чтобы помимо выхода с пуллер-слоя использовался эмбеддинг cls-токена с последнего слоя. -- 3 балла

In [20]:
class SentimentClassifier2(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    self.bert = BertModel.from_pretrained(MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size*2, n_classes)
  
  def forward(self, input_ids, attention_mask, labels):
    last_hidden_state, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    cls_token = last_hidden_state[:,0,:] #cls это первый токен
    stacked = torch.cat((cls_token, pooled_output), 1) #соединяем
    output = self.drop(stacked)
    return (loss_func(self.out(output), labels), self.out(output))

In [21]:
model2 = SentimentClassifier2(2)
model2 = model2.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbaf

обучаем модель 2

In [22]:
trainer2 = Trainer(
    model=model2,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics,    # metrics to evaluate
    data_collator=data_collator
)

trainer2.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.230100,0.478896,0.776000,0.800712,0.721154,0.900000
2,0.262400,0.409968,0.847000,0.840459,0.877996,0.806000
3,0.083400,0.538722,0.847000,0.846231,0.850505,0.842000
4,0.167800,0.657852,0.845000,0.839710,0.869379,0.812000
5,0.073600,0.685165,0.845000,0.840041,0.867804,0.814000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1250, training_loss=0.2541024965673685, metrics={'train_runtime': 435.6197, 'train_samples_per_second': 22.956, 'train_steps_per_second': 2.869, 'total_flos': 0.0, 'train_loss': 0.2541024965673685, 'epoch': 5.0})

Результаты на валидационной выборке

In [23]:
trainer2.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.845,
 'eval_f1': 0.8400412796697626,
 'eval_loss': 0.6851654052734375,
 'eval_precision': 0.8678038379530917,
 'eval_recall': 0.814,
 'eval_runtime': 13.0177,
 'eval_samples_per_second': 76.818,
 'eval_steps_per_second': 9.602}

Результаты на тестовой выборке

In [24]:
trainer2.evaluate(eval_dataset=test_dataset, 
                 metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 5.0,
 'test_accuracy': 0.862,
 'test_f1': 0.8580246913580246,
 'test_loss': 0.5987787246704102,
 'test_precision': 0.8834745762711864,
 'test_recall': 0.834,
 'test_runtime': 12.9711,
 'test_samples_per_second': 77.094,
 'test_steps_per_second': 9.637}

##примените к данным готовую модель для классификации последовательности (типа BertForSequenceClassification) -- 2 балла

In [25]:
model3 = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model3 = model3.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbaf

In [26]:
trainer3 = Trainer(
    model=model3,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics,    # metrics to evaluate
    data_collator=data_collator
)

trainer3.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.216600,0.435869,0.804000,0.819188,0.760274,0.888000
2,0.255200,0.413888,0.844000,0.841463,0.855372,0.828000
3,0.071600,0.558932,0.838000,0.831601,0.865801,0.800000
4,0.235700,0.623796,0.848000,0.843943,0.867089,0.822000
5,0.092700,0.657780,0.852000,0.847423,0.874468,0.822000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1250, training_loss=0.24355778407156467, metrics={'train_runtime': 435.7315, 'train_samples_per_second': 22.95, 'train_steps_per_second': 2.869, 'total_flos': 395496714240000.0, 'train_loss': 0.24355778407156467, 'epoch': 5.0})

Результаты на валидационной выборке

In [27]:
trainer3.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.852,
 'eval_f1': 0.8474226804123711,
 'eval_loss': 0.6577804684638977,
 'eval_precision': 0.874468085106383,
 'eval_recall': 0.822,
 'eval_runtime': 13.095,
 'eval_samples_per_second': 76.365,
 'eval_steps_per_second': 9.546}

Результаты на тестовой выборке

In [28]:
trainer3.evaluate(eval_dataset=test_dataset, 
                 metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 5.0,
 'test_accuracy': 0.864,
 'test_f1': 0.859504132231405,
 'test_loss': 0.5897068381309509,
 'test_precision': 0.8888888888888888,
 'test_recall': 0.832,
 'test_runtime': 12.9748,
 'test_samples_per_second': 77.072,
 'test_steps_per_second': 9.634}

##выберите на сайте google play три понравившихся вам отзыва, относящиеся к разным классам. Покажите, как на них работает любая из обученных моделей -- 2 балла


Ответ модели: 0 - положительный отзыв, 1 - негативный

In [29]:
positive = "A great movie that had me laughing out loud and crying and back and forth and together. Very poignant reminder showing what we put our troops through. They deserve better treatment from everyone including those brilliant dogs that are born and trained to go fight and save lives. Don't ever underestimate what training can do for a dog and don't give up on them, please."
positive2 = "I LOVE this movie! A fantastic mix of love, pain & seeing deeper in characters from the first movie. My kids, who are extremely picky with their music, and movies, LOVED this one. We've paid over 200$ to see this movie over and over! That was the only flaw. You couldn't buy it. But it makes sense for the cast and crew, they deserve the money they make from this movie!! We are happy to contribute, such a pleasure. Thank you SING 2 cast and crew! We have been enjoying your singing 24/7"
negative = "This movie was boring and the lead unlikable. I walked out less than halfway through. I'm glad I didn't pay to see this trash. A24, I'm disappointed in you. Stick to making horror movies."

положительный отзыв 1 (ответ правильный)

In [33]:
tokenized = tokenizer.encode_plus(positive, 
                          truncation=True,
                          padding=True,
                          return_token_type_ids=False,
                          return_attention_mask=True,
                          max_length=512,
                          return_tensors='pt'
                          )
tokenized = tokenized.to(device)

In [34]:
model3(input_ids=tokenized['input_ids'], attention_mask=tokenized['attention_mask'])[0].cpu().detach().numpy().argmax()

1

положительный отзыв 2 (ответ правильный)

In [35]:
tokenized = tokenizer.encode_plus(positive2, 
                          truncation=True,
                          padding=True,
                          return_token_type_ids=False,
                          return_attention_mask=True,
                          max_length=512,
                          return_tensors='pt'
                          )
tokenized = tokenized.to(device)

In [36]:
model3(input_ids=tokenized['input_ids'], attention_mask=tokenized['attention_mask'])[0].cpu().detach().numpy().argmax()

1

негативный отзыв (ответ правильный)

In [37]:
tokenized = tokenizer.encode_plus(negative, 
                          truncation=True,
                          padding=True,
                          return_token_type_ids=False,
                          return_attention_mask=True,
                          max_length=512,
                          return_tensors='pt'
                          )
tokenized = tokenized.to(device)

In [38]:
model3(input_ids=tokenized['input_ids'], attention_mask=tokenized['attention_mask'])[0].cpu().detach().numpy().argmax()

0

##агрегируйте cls-токены для нескольких слоев, чтобы сделать предсказание класса -- 2 дополнительных балла

In [42]:
class SentimentClassifier4(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    self.bert = BertModel.from_pretrained(MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size*2, n_classes)
  
  def forward(self, input_ids, attention_mask, labels):
    last_hidden_state, pooled_output, hidden_states = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      output_hidden_states=True,
      return_dict=False)
    cls_tokens =  torch.stack(([(i[:,0,:])for i in hidden_states])) #возьмем эмбединги CLS токенов с каждого слоя
    cls_tokens_mean = torch.mean(cls_tokens, 0) # усредним
    stacked = torch.cat((cls_tokens_mean, pooled_output), 1)
    output = self.drop(stacked)
    return (loss_func(self.out(output), labels), self.out(output))

In [43]:
model4 = SentimentClassifier4(2)
model4 = model4.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbaf

In [44]:
trainer4 = Trainer(
    model=model4,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics,    # metrics to evaluate
    data_collator=data_collator
)

trainer4.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.254700,0.429589,0.805000,0.811958,0.783985,0.842000
2,0.234700,0.419113,0.835000,0.830072,0.855626,0.806000
3,0.039800,0.548975,0.838000,0.836364,0.844898,0.828000
4,0.283700,0.661595,0.850000,0.842437,0.887168,0.802000
5,0.095400,0.682827,0.843000,0.836968,0.870410,0.806000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1250, training_loss=0.2550277123719454, metrics={'train_runtime': 437.7481, 'train_samples_per_second': 22.844, 'train_steps_per_second': 2.856, 'total_flos': 0.0, 'train_loss': 0.2550277123719454, 'epoch': 5.0})

Результаты на валидационой выборке

In [45]:
trainer4.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.843,
 'eval_f1': 0.8369678089304257,
 'eval_loss': 0.6828269362449646,
 'eval_precision': 0.8704103671706264,
 'eval_recall': 0.806,
 'eval_runtime': 13.1612,
 'eval_samples_per_second': 75.981,
 'eval_steps_per_second': 9.498}

Результаты на тестовой выборке

In [46]:
trainer4.evaluate(eval_dataset=test_dataset, 
                 metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'epoch': 5.0,
 'test_accuracy': 0.852,
 'test_f1': 0.8442105263157895,
 'test_loss': 0.6106184124946594,
 'test_precision': 0.8911111111111111,
 'test_recall': 0.802,
 'test_runtime': 13.1171,
 'test_samples_per_second': 76.236,
 'test_steps_per_second': 9.53}